In [1]:
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

### A/B тестирование

В этом задании вы познакомитесь с A/B тестированием и примените полученные знания по статистике. 

Рассмотрим A/B тестирование на примере сайта. У сайта есть два дизайна - старый и новый, и мы хотим оценить, насколько новый дизайн лучше старого. Для этого пользователи сайта случайным образом разделяются на контрольную и тестовую группы. Контрольной группе показывается старая версия сайта, тестовой группе - измененная версия. Оценить изменение можно несколькими способами, самый простой - оценить конверсию. Конверсия - доля пользователей, совершивших заранее определенное действие(например подписка, нажатие на кнопку, заполнение формы).

### Описание данных

Для начала нужно загрузить данные из файла `a_b_testing.csv` при помощи функции `read_csv` из библиотеки `pandas`. В данном случае 1 - была совершена подписка на сайт, 0 - подписки не было. A - контрольная группа, B - тестовая группа.

Далее нужно выполнить следующие пункты, описание выходного формата содержится внутри каждого задания.

### Доверительный интервал

В видео про доверительный интервал мы рассмотрели, как можно аппроксимировать биномиальное распределение нормальным. В некоторых случаях параметры нормального распределения можно вывести математически и ниже мы рассмотрим как.

Представим количество пользователей как случайную величину из биномиального распределения с параметрами  `n`  - количество пользователей и `p` - вероятность конверсии или как сумму  `n`  независимых бросаний монетки. Определим следующую случайную величину:

$$Y = X_{1} + X_{2} + \dots + X_{n} , \, $$
где случайная величина $X_{i}$ имеет распределение Бернулли. Для случайной величины $Y$ математическое ожидание и дисперсия равны:

$$\mu = np, \, \sigma^{2} = np\cdot(1 - p)$$

Далее применяя центральную предельную теорему(случайные величины $X_{i}$ распределены независимо и размер выборки большой), получаем что 

$$Y \sim \mathcal{N}(np \, np\cdot(1 - p))\$$

Мы перешли от биномиального распределения к нормальному. Следующий шаг - стандартизация нормального распределения:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} \sim \mathcal{N}(0, \, 1) $$ 

Преобразуем выражение выше:

$$Z = \frac{Y - np}{\sqrt{np\cdot(1-p)}} = \frac{\frac{Y}{n} - p}{\sqrt{\frac{p(1-p)}{n}}} \sim \mathcal{N}(0, \, 1) $$

Так как среднее значение по выборке - это наблюдаемый процент конверсии, то доверительный интервал будет выглядеть следующим образом:

$${P}\left(p - z_{1-\frac{\alpha}{2}} \sqrt{\frac{p(1-p)}{n}} \le \mu \le p + z_{1-\frac{\alpha}{2}}\sqrt{\frac{p(1-p)}{n}}\right) = 1-\alpha$$

### ЗАДАНИЕ

Найдите доверительный интервал для средней конверсии пользователей из контрольной выборки с уровнем значимости 95%. Округлите левую и правую границу с точностью до двух знаков после запятой. Запишите значения левой и правой границ через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой.

#### РЕШЕНИЕ

In [2]:
data = pd.read_csv('a_b_testing.csv')
b = data[data['group'] =='A']['converted']

In [3]:
#!pip install statsmodels
import statsmodels.stats.api as sms 
sms.DescrStatsW(b).tconfint_mean() 

(0.3904889829710936, 0.43327144903676107)

In [4]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h
mean_confidence_interval(b)

(0.4118802160039273, 0.3904889829710935, 0.4332714490367611)

### Задача A/B тестирования

Рассмотрим независимые выборки $X$ и $Y$ для которых есть $\mu_x$ и $\mu_y$, определяющие среднее значение распределения.

Рассматривается следующая гипотеза:
$$
H_0: \mu_x = \mu_y
$$
против альтернативы:

$$
H_1: \mu_x \ne \mu_y.
$$

Если гипотеза $H_0$ отвергается, то показатель действительно поменялся.

Также можно тест можно записать и другим способом:
$$
H_0: \mu_x \le \mu_y
$$

против альтернативы:

$$
H_1: \mu_x > \mu_y
$$

### Задание по статистике Стьюдента 

Найдите значение статистики Стьюдента в предположении независимости выборок по формуле:

$$
T(X, Y) = \frac{\bar{X} - \bar{Y}}{\sqrt{\frac{s_x^2}{n} + \frac{s_y^2}{m}}}
$$

где `n` - размер контрольной выборки, `m`  - размер тестовой выборки.

Ответ запишите в переменную `answer2` с точностью до двух знаков после запятой.

### РЕШЕНИЕ

In [5]:
ctrl = data[data['group'] == 'A']['converted'] 
test = data[data['group'] == 'B']['converted'] 

In [6]:
t = (np.mean(ctrl) - np.mean(test)) / np.sqrt(
    ((np.std(ctrl, ddof = 1) ** 2) / ctrl.size) + ((np.std(test, ddof = 1) ** 2) / test.size )
)
t

-1.6123875817242772

### Статистика Стьюдента из библиотеки Scipy

Найдите p-value для статистики Стьюдента, используя функцию `stats.ttest_ind`.

### РЕШЕНИЕ

In [7]:
from scipy.stats import ttest_ind

t = ttest_ind(ctrl, test, equal_var=False)
t

Ttest_indResult(statistic=-1.6123875817242772, pvalue=0.10695671235201251)

Дополнительная проверка: значение статистики Стьюдента, посчитанная двумя способами, должны совпадать.

Ответ запишите в переменную `answer3` с точностью до 2 знака после запятой